In [1]:
import os
print(os.getcwd())

/


In [2]:
# ✅ Define base project path
base_path = '/flickr_logo_yolo_project/'

# ✅ Define subfolders
image_dir = f'{base_path}images/train'
label_dir = f'{base_path}labels/train'
annotation_file = f'{base_path}annotations/flickr_logos_27_dataset_training_set_annotation.txt'
model_weights = f'{base_path}yolov8s.pt'
yaml_file = f'{base_path}logo_dataset.yaml'
output_dir = f'{base_path}yolo_output'

In [3]:
import os
from PIL import Image
from pathlib import Path

# Ensure output folder exists
os.makedirs(label_dir, exist_ok=True)

# Load annotations
with open(annotation_file, 'r') as f:
    lines = f.readlines()

# Generate class-to-ID map
logo_classes = sorted(list(set([line.split()[1] for line in lines])))
class_to_id = {cls_name: idx for idx, cls_name in enumerate(logo_classes)}

# Conversion helper
def convert_bbox(image_width, image_height, x1, y1, x2, y2):
    x_center = ((x1 + x2) / 2) / image_width
    y_center = ((y1 + y2) / 2) / image_height
    bbox_width = (x2 - x1) / image_width
    bbox_height = (y2 - y1) / image_height
    return x_center, y_center, bbox_width, bbox_height

# Generate label files
for line in lines:
    parts = line.strip().split()
    if len(parts) != 7:
        continue
    img_name, class_name, _, x1, y1, x2, y2 = parts
    img_path = os.path.join(image_dir, img_name)

    if not os.path.exists(img_path):
        continue

    with Image.open(img_path) as img:
        w, h = img.size

    x1, y1, x2, y2 = map(int, [x1, y1, x2, y2])
    x_c, y_c, bw, bh = convert_bbox(w, h, x1, y1, x2, y2)

    label_path = os.path.join(label_dir, Path(img_name).stem + '.txt')
    with open(label_path, 'w') as f:
        f.write(f"{class_to_id[class_name]} {x_c:.6f} {y_c:.6f} {bw:.6f} {bh:.6f}\n")

In [4]:
import yaml

yaml_content = {
    'train': image_dir,
    'val': image_dir,  # using train set for now
    'nc': len(class_to_id),
    'names': list(class_to_id.keys())
}

with open(yaml_file, 'w') as f:
    yaml.dump(yaml_content, f)

print(f"✅ YAML written to {yaml_file}")

✅ YAML written to /flickr_logo_yolo_project/logo_dataset.yaml


In [5]:
import sys
!{sys.executable} -m pip install ultralytics

In [6]:
from ultralytics import YOLO

model = YOLO(model_weights)

model.train(
    data=yaml_file,
    epochs=30,
    imgsz=416,
    batch=16,
    name='logo_detector_yolov8',
    project='.'  # Save results in current directory
)

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Ultralytics 8.3.161 🚀 Python-3.12.11 torch-2.7.1+cu128 CUDA:0 (NVIDIA GeForce RTX 5090, 32120MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/flickr_logo_yolo_project/logo_dataset.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=30, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=416, int8=False, i

100%|██████████| 755k/755k [00:00<00:00, 12.7MB/s]

Overriding model.yaml nc=80 with nc=27

                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     29056  ultralytics.nn.modules.block.C2f             [64, 64, 1, True]             
  3                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  4                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  5                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  6                  -1  2    788480  ultralytics.nn.modules.block.C2f             [256, 256, 2, True]           
  7                  -1  1   1180672  ultralytic

Model summary: 129 layers, 11,146,049 parameters, 11,146,033 gradients, 28.7 GFLOPs

Transferred 349/355 items from pretrained weights
Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks...


100%|██████████| 5.35M/5.35M [00:00<00:00, 52.3MB/s]


...

                   all       1079        809      0.921      0.957      0.969      0.815



30 epochs completed in 0.048 hours.
Optimizer stripped from logo_detector_yolov8/weights/last.pt, 22.5MB
Optimizer stripped from logo_detector_yolov8/weights/best.pt, 22.5MB

Validating logo_detector_yolov8/weights/best.pt...
Ultralytics 8.3.161 🚀 Python-3.12.11 torch-2.7.1+cu128 CUDA:0 (NVIDIA GeForce RTX 5090, 32120MiB)
Model summary (fused): 72 layers, 11,136,033 parameters, 0 gradients, 28.5 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 34/34 [00:02<00:00, 14.30it/s]


                   all       1079        809      0.921      0.957      0.969      0.815
                Adidas         30         30      0.854        0.9      0.975       0.78
                 Apple         30         30      0.992          1      0.995       0.82
                   BMW         30         30      0.955          1      0.992      0.843
               Citroen         30         30      0.927      0.967       0.99      0.839
              Cocacola         29         29      0.898      0.912      0.949       0.81
                   DHL         30         30       0.98          1      0.995      0.809
                 Fedex         30         30      0.909          1      0.991      0.811
               Ferrari         30         30      0.926          1      0.994      0.829
                  Ford         30         30      0.951          1      0.995       0.87
                Google         30         30      0.926      0.967      0.962      0.839
                    H

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7fc134ca78c0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.0

In [7]:
# Load annotation lines again
annotation_file = '/flickr_logo_yolo_project/annotations/flickr_logos_27_dataset_training_set_annotation.txt'
with open(annotation_file, 'r') as f:
    lines = f.readlines()

# Rebuild class mapping
logo_classes = sorted(list(set([line.split()[1] for line in lines])))
class_to_id = {cls_name: idx for idx, cls_name in enumerate(logo_classes)}

In [8]:
import pickle

# Define what to save
model_metadata = {
    'weights_path': '/logo_detector_yolov8/weights/best.pt',
    'class_to_id': class_to_id
}

# Save as .pkl
with open('/flickr_logo_yolo_project/yolo_model_metadata.pkl', 'wb') as f:
    pickle.dump(model_metadata, f)

print("✅ Model metadata and class map saved as .pkl")

✅ Model metadata and class map saved as .pkl


In [27]:
from PIL import Image
from ultralytics import YOLO
import cv2
import matplotlib.pyplot as plt
import os

def preprocess_and_predict(image_path, model_path, save_path="/flickr_logo_yolo_project/predicted_images/", conf=0.3):
    """
    Preprocesses a test image to match YOLOv8 training style and runs prediction.
    - Resizes to 416x416
    - Converts to RGB
    - Runs YOLOv8 prediction
    - Saves and displays result
    """

    # Load model
    model = YOLO(model_path)

    # Preprocess image: RGB and resize to 416x416
    img = Image.open(image_path).convert("RGB").resize((416, 416))
    temp_path = "temp_resized.jpg"
    img.save(temp_path)

    # Predict on the resized image
    results = model.predict(source=temp_path, conf=conf, imgsz=416)

    # Visualize and save output
    result_img = results[0].plot()
    os.makedirs(os.path.dirname(save_path), exist_ok=True)
    cv2.imwrite(save_path, result_img)

    # Show inline
    plt.imshow(cv2.cvtColor(result_img, cv2.COLOR_BGR2RGB))
    plt.axis("off")
    plt.title("Prediction")
    plt.show()

    return results[0]

In [28]:
model_path = "logo_detector_yolov8/weights/best.pt"
test_image = "/flickr_logo_yolo_project/test_images/WhatsApp Image 2025-06-30 at 20.11.55.jpeg" 
preprocess_and_predict(test_image, model_path)


image 1/1 /temp_resized.jpg: 416x416 (no detections), 5.6ms
Speed: 0.9ms preprocess, 5.6ms inference, 0.6ms postprocess per image at shape (1, 3, 416, 416)


error: OpenCV(4.11.0) /io/opencv/modules/imgcodecs/src/loadsave.cpp:836: error: (-2:Unspecified error) could not find a writer for the specified extension in function 'imwrite_'
